In [30]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib
bucket_name = 'sagemaker-studio-42jgegdgih'

In [31]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train a logistic regression model with increased max_iter
model = LogisticRegression(max_iter=1000)  # Increase the number of iterations
model.fit(X_train_scaled, y_train)  # Use scaled data

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
joblib.dump(model, 'model.joblib')

['model.joblib']

In [32]:
import boto3

client = boto3.client('iam')

response = client.list_roles()

#for role in response['Roles']:
   #print("Role name: {}, ARN: {}".format(role['RoleName'], role['Arn']))
    
role = 'arn:aws:iam::870665956147:role/service-role/AmazonSageMaker-ExecutionRole-20231023T114118'

In [33]:
from sagemaker import Model, Session
from sagemaker.image_uris import retrieve
import boto3

# Initialize variables
model_data = f's3://{bucket_name}/model/model.tar.gz'

# Initialize the SageMaker Session
sagemaker_session = Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [14]:
model_data

's3://sagemaker-studio-42jgegdgih/model/model.tar.gz'

In [23]:
# Get the Docker image URI for the Scikit-Learn framework
image_uri = retrieve('sklearn', region='us-east-2', version='0.23-1')

# Create a Model object
model = Model(
    image_uri=image_uri,
    model_data=model_data,
    role=role,
    sagemaker_session=sagemaker_session
)

In [ ]:
# Create a Model Package Group
model_package_group_name = "MyModelPackageGroup2"
model_package_group_input_dict = {
    'ModelPackageGroupName': model_package_group_name,
    'ModelPackageGroupDescription': 'My model package group'
}
sagemaker_session.sagemaker_client.create_model_package_group(**model_package_group_input_dict)

{'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-2:870665956147:model-package-group/MyModelPackageGroup2',
 'ResponseMetadata': {'RequestId': '54fb9a2e-1123-41e8-845c-f537640b8980',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '54fb9a2e-1123-41e8-845c-f537640b8980',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '108',
   'date': 'Wed, 01 Nov 2023 12:21:03 GMT'},
  'RetryAttempts': 0}}

In [17]:

# Model Package configurations
model_package_input_dict = {
    'ModelPackageDescription': 'My model package description',
    'ModelPackageGroupName': model_package_group_name,
    'InferenceSpecification': {
        'Containers': [
            {
                'Image': image_uri,
                'ModelDataUrl': model_data
            },
        ],
        'SupportedContentTypes': ['application/jsonlines'],
        'SupportedResponseMIMETypes': ['application/jsonlines']
    }
}

In [18]:
# Register the Model
model_package_response = sagemaker_session.sagemaker_client.create_model_package(**model_package_input_dict)

print(f"Model Package ARN: {model_package_response['ModelPackageArn']}")

response = sagemaker_session.sagemaker_client.list_model_packages(ModelPackageGroupName='MyModelPackageGroup1')

for model_package in response['ModelPackageSummaryList']:
    print(f"Model package ARN: {model_package['ModelPackageArn']}")


Model Package ARN: arn:aws:sagemaker:us-east-2:870665956147:model-package/MyModelPackageGroup2/1
Model package ARN: arn:aws:sagemaker:us-east-2:870665956147:model-package/MyModelPackageGroup1/1


In [ ]:
#Now track the experiment with the same model to register the accuracy

In [28]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [29]:
experiment_name = "Iris-Experiment"
experiment_description = "Experiment to track Iris dataset models"
iris_experiment = Experiment.create(
    experiment_name=experiment_name,
    description=experiment_description,
    sagemaker_boto_client=sagemaker_session.sagemaker_client
)

In [35]:
import time

trial_name = f"iris-trial-{int(time.time())}"  # Ensuring a unique name using a timestamp
iris_trial = Trial.create(
    trial_name=trial_name,
    experiment_name=iris_experiment.experiment_name,
    sagemaker_boto_client=sagemaker_session.sagemaker_client
)


In [36]:
with Tracker.create(sagemaker_boto_client=sagemaker_session.sagemaker_client) as tracker:
    tracker.log_parameter("max_iter", 1000)  # Example of logging a parameter
    tracker.log_metric("accuracy", accuracy)  # Logging the accuracy
    
# Associate the trial component with the trial
iris_trial.add_trial_component(tracker.trial_component)


In [38]:
iris_trial

Trial(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7ff87f90f5b0>,trial_name='iris-trial-1698929046',experiment_name='Iris-Experiment',tags=None,trial_arn='arn:aws:sagemaker:us-east-2:870665956147:experiment-trial/iris-trial-1698929046',response_metadata={'RequestId': '36a5cbb8-85cc-4af9-b53f-e4a69ea5cc59', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '36a5cbb8-85cc-4af9-b53f-e4a69ea5cc59', 'content-type': 'application/x-amz-json-1.1', 'content-length': '94', 'date': 'Thu, 02 Nov 2023 12:44:06 GMT'}, 'RetryAttempts': 0})